## 4D Objects By Change - Creating an analysis

This notebook explains the data preparation step for the extraction of *4D Objects-By-Change* (4D-OBCs; [Anders et al., 2020](#References)). For details about the method, we refer to the articles by Anders et al. ([2020; 2021](#References)).

In [ ]:
import py4dgeo
import numpy as np

The main data structure for the 4D-OBC algorithm is a `SpatiotemporalAnalysis` object. It is always backed by an archive file, which we specify when instantiating the analysis object. If the given filename already exists, we open it, otherwise an empty archive is created:

In [ ]:
analysis = py4dgeo.SpatiotemporalAnalysis("test.zip", force=True)

The analysis object stores all information necessary to (re-)run and extend the analysis. This includes, e.g., the reference epoch, the core points, and the space-time array of change values. However, it does not store all epochs that were used in building up the analysis. 

In the following we show how the required components are added to the analysis using the example of the two epochs used for [demonstration of the M3C2](m3c2.ipynb). To be usable during spatiotemporal segmentation, a timestamp is added to these epochs, respectively:

In [ ]:
reference_epoch, epoch = py4dgeo.read_from_xyz(
    "plane_horizontal_t1.xyz", "plane_horizontal_t2.xyz"
)
reference_epoch.timestamp = "March 9th 2022, 16:00"
epoch.timestamp = "March 10th 2022, 16:00"

The reference epoch needs to be added as such to the analysis:

In [ ]:
analysis.reference_epoch = reference_epoch

We also need to define the set of core points in use. In this example, we use all points in the reference epoch as core points:

In [ ]:
analysis.corepoints = reference_epoch

Next, we want to add epochs to the spatiotemporal analysis. For this, we need to define a method that calculates the point cloud distances, for example using the M3C2 algorithm ([Lague et al., 2013](#references)). For details of the algorithm setup, you can have a look at the [M3C2 notebook](m3c2.ipynb).

In [ ]:
analysis.m3c2 = py4dgeo.M3C2(cyl_radii=[2.0], normal_radii=[2.0])

Having done all of this, we can add an epoch to the analysis:

In [ ]:
analysis.add_epochs(epoch)

Note that the `add_epochs` can be passed several epochs at the same time to save some costly memory operations: `add_epochs(*list_of_epochs)`. Also, `add_epochs` can be called again on existing analysis objects allowing you to update your analysis after additional data acquisition at the same site.

Having done this, we can inspect some of the data contained in the analysis:

In [ ]:
np.set_printoptions(threshold=5)

In [ ]:
print(f"Space-time distance array: {analysis.distances}")

In [ ]:
print(f"Uncertainty array of M3C2 calculation: {analysis.uncertainties}")

In [ ]:
print(f"Timestamp deltas of analysis: {analysis.timedeltas}")

Note that the `add_epochs` can be passed several epochs at the same time to save some costly memory operations. Also, `add_epochs` can be called again on existing analysis objects allowing you to update your analysis after additional data acquisition at the same site.

Sometimes, you will want to apply preprocessing in the form of smoothing to the calculated distance array. You can do so by setting the analysis object's `smoothed_distances` property. When this was set, the 4D-OBC algorithm will operate on the smoothed data instead of the raw data stored in the analysis object. `py4dgeo` provides a smoothing implementation using a sliding window approach:

In [ ]:
analysis.smoothed_distances = py4dgeo.temporal_averaging(
    analysis.distances, smoothing_window=24
)

### References

* Anders, K., Winiwarter, L., Lindenbergh, R., Williams, J. G., Vos, S. E., & Höfle, B. (2020). 4D objects-by-change: Spatiotemporal segmentation of geomorphic surface change from LiDAR time series. ISPRS Journal of Photogrammetry and Remote Sensing, 159, pp. 352-363. doi: [10.1016/j.isprsjprs.2019.11.025](https://doi.org/10.1016/j.isprsjprs.2019.11.025).
* Anders, K., Winiwarter, L., Mara, H., Lindenbergh, R., Vos, S. E., & Höfle, B. (2021). Fully automatic spatiotemporal segmentation of 3D LiDAR time series for the extraction of natural surface changes. ISPRS Journal of Photogrammetry and Remote Sensing, 173, pp. 297-308. doi: [10.1016/j.isprsjprs.2021.01.015](https://doi.org/10.1016/j.isprsjprs.2021.01.015).
* Lague, D., Brodu, N., & Leroux, J. (2013). Accurate 3D comparison of complex topography with terrestrial laser scanner: Application to the Rangitikei canyon (N-Z). ISPRS Journal of Photogrammetry and Remote Sensing, 82, pp. 10-26. doi: [10.1016/j.isprsjprs.2013.04.009](https://doi.org/10.1016/j.isprsjprs.2013.04.009).